In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [20]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [5]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [7]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [8]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [10]:
# prepare embedding matrix  
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense, Dropout, LSTM, Bidirectional
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
#prepare Embedding layer
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')                            ## Input layer
embedded_sequences = embedding_layer(sequence_input)                                           ## Embeddings layer
X = Bidirectional(LSTM(200, return_sequences=False), merge_mode='concat')(embedded_sequences)  ## BiLSTM layer
X = Dropout(0.5)(X)                                                                            ## Dropout layer  
X = Dense(30, activation='relu')(X)                                                            ## Fully Connected layer
X = Dropout(0.5)(X)                                                                            ## Dropout layer
preds = Dense(2, activation='softmax')(X)                                                      ## Softmax layer
model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='/content/drive/My Drive/Sarcasm Detection/Plots/BILSTM.png')

In [40]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
5002/5002 [==============================] - 18s 4ms/step - loss: 0.3879 - acc: 0.8235
Epoch 2/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.2493 - acc: 0.8990
Epoch 3/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.2029 - acc: 0.9152
Epoch 4/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.1595 - acc: 0.9350
Epoch 5/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.1321 - acc: 0.9448
Epoch 6/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.1119 - acc: 0.9556
Epoch 7/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0825 - acc: 0.9714
Epoch 8/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0691 - acc: 0.9734
Epoch 9/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0690 - acc: 0.9760
Epoch 10/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0456 - acc: 0.9838
Epoch 11/15
5002/50

In [41]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

1667/1667 [==============================] - 3s 2ms/step
test loss is: 0.522789328958533
test accuracy is:  0.9130173965206959


In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    
    def recall(y_true, y_pred):    
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives      = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision           = true_positives / (predicted_positives + K.epsilon())
        return precision
      
    precision = precision(y_true, y_pred)
    recall    = recall(y_true, y_pred)
    f1        = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    return f1

model.compile(loss='categorical_crossentropy',
          optimizer= "rmsprop",
          metrics=[f1])

In [42]:
model.fit(x_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0250 - acc: 0.9934
Epoch 2/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0173 - acc: 0.9942
Epoch 3/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0598 - acc: 0.9926
Epoch 4/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0419 - acc: 0.9950
Epoch 5/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0135 - acc: 0.9956
Epoch 6/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0110 - acc: 0.9968
Epoch 7/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0177 - acc: 0.9952
Epoch 8/15
5002/5002 [==============================] - 17s 3ms/step - loss: 0.0120 - acc: 0.9952
Epoch 9/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0067 - acc: 0.9972
Epoch 10/15
5002/5002 [==============================] - 16s 3ms/step - loss: 0.0193 - acc: 0.9950
Epoch 11/15
5002/50

In [43]:
loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

1667/1667 [==============================] - 2s 1ms/step
test loss is: 0.8286889967750662
test f1 score is:  0.9154169166166767


In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 34)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 34, 300)           6178800   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 400)               801600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                12030     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 62        
Total para